# Classifier

meer grid based dingen toevoegen, nu gaan we niet hoger komen


In [143]:
from SimpleCV import *
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import sklearn 
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import cv2
import glob
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm


In [144]:
# df = pd.read_csv("../dataset-numpy/dataset_analysis.csv")
# wordt niet hoger door normalizen
df = pd.read_csv("../dataset-numpy/dataset_analysis_normalized.csv")
df.head()


# split df in data (X) and labels (y)
X, y = df.iloc[:,:-1], df.iloc[:,-1]

# create train and test data and labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.20, random_state=42)

In [145]:
min_max_scaler = preprocessing.MinMaxScaler() # Scale the training features
training_set = min_max_scaler.fit_transform(X_train)
# Scale the test features (using the same settings as the trainingset)
test_set = min_max_scaler.transform(X_test)

In [146]:
#Naive Bayes Gaussian
gnb = GaussianNB()
gnb.fit(training_set, y_train)
#y_pred = gnb.predict(X_test)
print "Gaussian :",gnb.score(test_set, y_test)

#Naive Bayes Bernoulli
bnb = BernoulliNB()
bnb.fit(training_set, y_train)
#y_pred = gnb.predict(X_test)
print "Bernoulli:",bnb.score(test_set, y_test)


Gaussian : 0.8515625
Bernoulli: 0.838541666667


In [184]:
#Random Forest
# Create a random forest classifier
clfRF = RandomForestClassifier(n_jobs=2, random_state=0)
# Train the classifier to take the training features and learn how they relate to the training y
clfRF.fit(training_set, y_train)
print "Random forest:",clfRF.score(test_set, y_test)

Random forest: 0.9296875


In [185]:
# #Random Forest
# # Create a random forest classifier
# clfRF = RandomForestClassifier(n_jobs=2, random_state=0)
# # Train the classifier to take the training features and learn how they relate to the training y
# clfRF.fit(training_set, y_train)
# print "Random forest:",clfRF.score(test_set, y_test)

# #Decision tree
# clfDT = DecisionTreeClassifier(random_state=0)
# clfDT.fit(training_set, y_train)
# print "Decision tree:",clfDT.score(test_set, y_test)

clfRF = RandomForestClassifier(bootstrap= False,  max_depth= 40, max_features= 'sqrt',
                               min_samples_leaf = 1, min_samples_split = 2, n_estimators = 1400)
# Train the classifier to take the training features and learn how they relate to the training y
clfRF.fit(training_set, y_train)
print "Random forest:",clfRF.score(test_set, y_test)
print(clfRF.get_params())

Random forest: 0.96875
{'warm_start': False, 'oob_score': False, 'n_jobs': 1, 'min_impurity_decrease': 0.0, 'verbose': 0, 'max_leaf_nodes': None, 'bootstrap': False, 'min_samples_leaf': 1, 'n_estimators': 1400, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'criterion': 'gini', 'random_state': None, 'min_impurity_split': None, 'max_features': 'sqrt', 'max_depth': 40, 'class_weight': None}


In [148]:
# KNN
# niet accurate, hoeveelheid neighbors:
# Score KNN:  1 0.971354166667
# Score KNN:  2 0.9609375
# Score KNN:  3 0.963541666667
# Score KNN:  4 0.966145833333
# Score KNN:  5 0.963541666667
# Score KNN:  6 0.963541666667
# Score KNN:  7 0.963541666667
# Score KNN:  8 0.9609375
# Score KNN:  9 0.963541666667

for i in range (1,10):
    # Create a classifier with k 
    clfKNN = KNeighborsClassifier(n_neighbors=i)
    # Train the classifier with the training data and training labels
    clfKNN.fit(training_set, y_train)

    # Score the classifier
    # (calculates the mean accuracy of the given test data) 
    score = clfKNN.score(test_set, y_test)
    print "Score KNN: ", i , score

Score KNN:  1 0.971354166667
Score KNN:  2 0.963541666667
Score KNN:  3 0.96875
Score KNN:  4 0.963541666667
Score KNN:  5 0.966145833333
Score KNN:  6 0.9609375
Score KNN:  7 0.963541666667
Score KNN:  8 0.958333333333
Score KNN:  9 0.9609375


# SVM with Gridsearch

In [189]:
parameters = {'kernel' : ('linear', 'rbf'), 'C': [1,10], 'gamma': np.arange(.1, .2, 0.01)}

# svc = svm.SVC(gamma = j)
# print j
clf = GridSearchCV(svm.SVC(), parameters, cv = 5)
clf.fit(X_train, y_train)

print clf.best_params_
print clf.best_score_

{'kernel': 'rbf', 'C': 10, 'random_state': None, 'gamma': 0.16999999999999998}
0.9765625
